# capacitated vehicle routing problem with time-windows

In [ ]:
import pandas as pd
from itertools import permutations
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
INSTANCE_PATH = '../data/sample.txt'
SOLUTION_PATH = '../data/sample_solution.txt'

METRICS_FACTOR = 1000

## Load DATA

In [ ]:
# Read data from txt

# get data area
with open(INSTANCE_PATH, 'r') as f:
    lines = [l.strip() for l in f.readlines()]
instance_name = lines[0]
vechicle_row = lines.index('VEHICLE')
cust_row = lines.index('CUSTOMER')
vechicle_area = range(vechicle_row+1, cust_row-1)
cust_area = range(cust_row+2, len(lines))

# read data as dataframe
vechicle_df = pd.read_csv(
    INSTANCE_PATH, 
    skiprows=lambda x: x not in vechicle_area, 
    delim_whitespace=True)
vechicle_df.columns = ['num', 'capacity']
cust_df = pd.read_csv(
    INSTANCE_PATH, 
    skiprows=lambda x: x not in cust_area, 
    delim_whitespace=True,
    header=None)
cust_df.columns = ['no', 'x', 'y', 'demand', 'start_time', 'end_time', 'service_time']
cust_df = cust_df.sort_values('no')

# set data type
cust_df['no'] = cust_df['no'].astype('int')
cust_df['x'] = cust_df['x'].astype('float')
cust_df['y'] = cust_df['y'].astype('float')
cust_df['demand'] = cust_df['demand'].astype('int')
cust_df['start_time'] = cust_df['start_time'].astype('float')
cust_df['end_time'] = cust_df['end_time'].astype('float')
cust_df['service_time'] = cust_df['service_time'].astype('float')
vechicle_df['num'] = vechicle_df['num'].astype(int)
vechicle_df['capacity'] = vechicle_df['capacity'].astype(int)

In [ ]:
# Plot data
vechicle_info = 'vechicle num=%d, capacity=%d' \
    % (vechicle_df.loc[0, 'num'], vechicle_df.loc[0, 'capacity'])
fig = px.scatter(
    cust_df, x='x', y='y', 
    hover_data=['demand', 'start_time', 'end_time', 'service_time'], 
    text=cust_df['no'],
    title='vechicle num=%d, capacity=%d' \
        % (vechicle_df.loc[0, 'num'], vechicle_df.loc[0, 'capacity']),
    width=800, height=800)
fig.update_layout(yaxis=dict(scaleanchor='x'))
fig.show()

## Make Model

In [ ]:
# Make data for model

data = {}
# distance_matrix
dists = pd.DataFrame(list(permutations(cust_df['no'], 2)), columns=['from', 'to'])
coord_map = cust_df.set_index('no')[['x', 'y']]
dists['diff_x'] = coord_map.loc[dists['from'], 'x'].values - coord_map.loc[dists['to'], 'x'].values
dists['diff_y'] = coord_map.loc[dists['from'], 'y'].values - coord_map.loc[dists['to'], 'y'].values
dists['distance'] = ((dists['diff_x']**2) + (dists['diff_y']**2))**(1/2)
data['distance_matrix'] = dists.pivot_table(index=['from'], columns=['to'], values=['distance'])
data['distance_matrix'] = data['distance_matrix'].fillna(0)
data['distance_matrix'] = (METRICS_FACTOR*data['distance_matrix']).astype(int)
data['distance_matrix'] = data['distance_matrix'].values.tolist()
# time_matrix
data['time_matrix'] = data['distance_matrix']
# time_windows
data['time_windows'] = (METRICS_FACTOR*cust_df[['start_time', 'end_time']]).astype(int)
data['time_windows'] = data['time_windows'].values.tolist()
# demands
data['demands'] = cust_df['demand'].values.tolist()
# service_time
data['service_time'] = (METRICS_FACTOR*cust_df['service_time']).astype(int)
data['service_time'] = data['service_time'].values.tolist()
# num_vehicles
data['num_vehicles'] = vechicle_df.loc[0, 'num']
# vehicle_capacities
data['vehicle_capacities'] = [vechicle_df.loc[0, 'capacity']]*data['num_vehicles']
# depot
data['depot'] = 0

In [ ]:
# Set model
manager = pywrapcp.RoutingIndexManager(
    len(data['distance_matrix']), int(data['num_vehicles']), int(data['depot']))
routing = pywrapcp.RoutingModel(manager)

In [ ]:
# Set cost of total distance
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [ ]:
# Set constraint of capacity
def demand_callback(from_index):
    """Returns the demand of the node."""
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]

demand_callback_index = routing.RegisterUnaryTransitCallback(
    demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # null capacity slack
    data['vehicle_capacities'],  # vehicle maximum capacities
    True,  # start cumul to zero
    'Capacity')

In [ ]:
# Set constraint of time window
def time_callback(from_index, to_index):
    """Returns the travel time between the two nodes + service time of from_node."""
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['time_matrix'][from_node][to_node] + data['service_time'][from_node]

dim_name = 'Time'
time_callback_index = routing.RegisterTransitCallback(time_callback)
routing.AddDimension(
    time_callback_index,
    max(sum(data['time_windows'], [])),  # allow waiting time
    max(sum(data['time_windows'], [])),  # maximum time per vehicle
    False,  # Don't force start cumul to zero.
    dim_name)
time_dimension = routing.GetDimensionOrDie(dim_name)
# add time window constraints for each location except depot.
for location_idx, time_window in enumerate(data['time_windows']):
    if location_idx == data['depot']:
        continue
    index = manager.NodeToIndex(location_idx)
    time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
# add time window constraints for each vehicle start node.
depot_idx = data['depot']
for vehicle_id in range(data['num_vehicles']):
    index = routing.Start(vehicle_id)
    time_dimension.CumulVar(index).SetRange(
        data['time_windows'][depot_idx][0],
        data['time_windows'][depot_idx][1])
for i in range(data['num_vehicles']):
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.Start(i)))
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.End(i)))

### Main function

In [ ]:
# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.time_limit.seconds = 30

In [ ]:
# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

In [ ]:
# Check whether solver has solution
assert solution, 'Solver did not find any solutions.'

### Visualization

In [ ]:
# Display the routes.

def get_results(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    results = {}
    capa_dim = routing.GetDimensionOrDie('Capacity')
    time_dim = routing.GetDimensionOrDie('Time')
    for vehicle_id in range(routing.vehicles()):
        index = routing.Start(vehicle_id)
        result = {
            'route': [manager.IndexToNode(index)], 
            'cost': [routing.GetFixedCostOfVehicle(vehicle_id)],
            'capa': [0],                                  
            'time': [0]
            }
        while not routing.IsEnd(index):
            pre_index = index
            index = solution.Value(routing.NextVar(index))
            result['route'].append(manager.IndexToNode(index))
            result['cost'].append(
                routing.GetArcCostForVehicle(pre_index, index, vehicle_id))
            result['capa'].append(
                capa_dim.GetTransitValue(pre_index, index, vehicle_id))          
            result['time'].append(
                time_dim.GetTransitValue(pre_index, index, vehicle_id))
        result['cost'] = pd.Series(result['cost']).cumsum().to_list()
        result['capa'] = pd.Series(result['capa']).cumsum().to_list()
        result['time'] = pd.Series(result['time']).cumsum().to_list()
        results[vehicle_id] = result
    return results

solution_results = get_results(solution, routing, manager)
print('Result of Route: No.(cost,capacity,time)')
for i, res in solution_results.items():
  route = res['route']
  costs = res['cost']
  capas = res['capa']
  times = res['time']
  vechicle_capa = vechicle_df.loc[0, 'capacity']
  text = [f'{l}({c},{d}/{vechicle_capa},{t})' 
          for l, c, d, t in zip(route, costs, capas, times)]
  text = ' -> '.join(text)
  print(f'Vehicle {i}: {text}')
print('Objective value:', solution.ObjectiveValue())

In [ ]:
# Make result dataframe
resulf_df = []
raw_df_map = cust_df.set_index('no')
for i, res in solution_results.items():
  res = pd.DataFrame.from_dict(res)
  if len(res)>2:
    res.insert(0, 'vechile', i)
    resulf_df.append(res)
resulf_df = pd.concat(resulf_df, axis=0)
resulf_df = resulf_df.reset_index(drop=True)
resulf_df.columns = ['vechicle', 'no', 'distance', 'capacity', 'time']
resulf_df[['distance', 'time']] /= METRICS_FACTOR
resulf_df = resulf_df.merge(cust_df, on=['no'], how='left')
resulf_df

In [ ]:
# Show results
print('Total number of vechile:', len(resulf_df['vechicle'].unique()))
print('Total distance:', resulf_df.groupby('vechicle').last()['distance'].sum())
print()
print('Other total metrics of each vechicle:')
display(resulf_df.groupby('vechicle').last()[['distance', 'capacity', 'time']])

In [ ]:
# Plot data
fig = px.line(
    resulf_df, x='x', y='y', 
    hover_data=resulf_df.columns, 
    text=resulf_df['no'],
    color='vechicle',
    width=800, height=800)
fig.update_layout(yaxis=dict(scaleanchor='x'))
fig.show()